<a href="https://colab.research.google.com/github/DerrickMga/2023Projects/blob/main/Audio_Transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request
import requests
import openai

app = Flask(__name__)

ACCESS_TOKEN = "your_meta_access_token"
PHONE_NUMBER_ID = "your_phone_number_id"

# Download audio file
def download_audio(media_id):
    url = f"https://graph.facebook.com/v15.0/{media_id}"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        audio_url = response.json().get("url")
        audio_response = requests.get(audio_url, headers=headers)
        return audio_response.content
    else:
        raise Exception(f"Error fetching audio: {response.json()}")

# Transcribe audio using Whisper
def transcribe_audio(audio_content):
    openai.api_key = "your_openai_api_key"
    response = openai.Audio.transcribe(
        model="whisper-1",
        file=audio_content,
        response_format="text"
    )
    return response["text"]

# Send response to WhatsApp
def send_whatsapp_message(recipient, message):
    url = f"https://graph.facebook.com/v15.0/{PHONE_NUMBER_ID}/messages"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}", "Content-Type": "application/json"}
    payload = {
        "messaging_product": "whatsapp",
        "to": recipient,
        "type": "text",
        "text": {"body": message}
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code != 200:
        print(f"Error sending message: {response.json()}")

@app.route('/webhook', methods=['POST'])
def webhook():
    data = request.json
    if "messages" in data["entry"][0]["changes"][0]["value"]:
        message = data["entry"][0]["changes"][0]["value"]["messages"][0]
        if message["type"] == "audio":
            media_id = message["audio"]["id"]
            sender = message["from"]

            # Process audio
            audio_content = download_audio(media_id)
            transcription = transcribe_audio(audio_content)

            # Send transcription back to user
            send_whatsapp_message(sender, transcription)

    return "OK", 200

if __name__ == '__main__':
    app.run(debug=True)
